# Compartilhamento de Estados entre Processos

Antes de partirmos para os detalhes sobre o tema, é bom relembrar que
o recomendado é não compartilhar nada ou o mínimo possível de recursos/memória/dados
entre Threads/Processos.

Isso por que quando recursos/memória/dados são compartilhados, tudo
se torna mais complexo e o gerenciamento de Threads/Processos deixará seu código
mais complicado.

Quando falamos em compartilhar "Estados" você deve lembrar dos conceitos
de Orientação a Objetos, onde Classes possuem estados (atributos) e métodos.

A ideia aqui não é trabalhar com objetos, mas com valores simples realizando o
Compartilhamento de Memória.

### Compartilhamento de Memória

Recomendado caso você precise compartilhar uma variável de valor único ou arrays
entre seus processos. Utilizando as classes:

- multiprocessing.Value;
- multiprocessing.Array.

Mas um detalhe à se mencionar é que os Arrays da classe Array são como os Arrays
da linguagem de programação C, onde os Arrays tem valores de tipo único contidos
na coleção, diferente dos Arrays Python que suportam todo tipo de dado dentro de si
simultaneamente.

In [4]:
import multiprocessing
import time


def funcao1(val: int | float, stat: bool) -> None:
    if stat:
        res = val + 10
        stat = False
    else:
        res = val + 20
        val = 200
        stat = True
    print(f"O resultado da funcao1 é: {res}")
    time.sleep(0.001)


def funcao2(val: int | float, stat: bool) -> None:
    if stat:
        res = val + 30
        stat = False
    else:
        res = val + 40
        val = 400
        stat = True
    print(f"O resultado da funcao2 é: {res}")
    time.sleep(0.001)


def main() -> None:
    valor = 100
    status = False

    pcs = [
        multiprocessing.Process(target=funcao1, args=(valor, status)),
        multiprocessing.Process(target=funcao2, args=(valor, status)),
    ]
    [pc.start() for pc in pcs]
    [pc.join() for pc in pcs]


if __name__ == "__main__":
    main()

O resultado da funcao2 é: 140
O resultado da funcao1 é: 120


In [5]:
import ctypes


def funcao1(val: int | float, stat: bool) -> None:
    if stat.value:
        res = val.value + 10
        stat.value = False
    else:
        res = val.value + 20
        val.value = 200
        stat.value = True
    print(f"O resultado da funcao1 é: {res}")
    time.sleep(0.001)


def funcao2(val: int | float, stat: bool) -> None:
    if stat.value:
        res = val.value + 30
        stat.value = False
    else:
        res = val.value + 40
        val.value = 400
        stat.value = True
    print(f"O resultado da funcao2 é: {res}")
    time.sleep(0.001)


def main() -> None:
    valor = multiprocessing.Value("i", 100)
    status = multiprocessing.Value(ctypes.c_bool, False)

    pcs = [
        multiprocessing.Process(target=funcao1, args=(valor, status)),
        multiprocessing.Process(target=funcao2, args=(valor, status)),
    ]
    [pc.start() for pc in pcs]
    [pc.join() for pc in pcs]


if __name__ == "__main__":
    main()

O resultado da funcao1 é: 120
O resultado da funcao2 é: 230
